In [1]:
#importing necessary libraries
import webbpsf
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from scipy.stats import norm
from scipy.optimize import curve_fit
import math
from IPython.display import clear_output

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-062 CANNOT BE CHECKED AGAINST ONLINE VERSION


In [2]:
def NoiseMap(path,writefile,bkgsubfile):
    def gauss_f(x,mu,sig):
        return (1/(sig*(np.sqrt(2*np.pi))))*np.exp((-1/2)*(((x-mu)**2)/(sig**2)))
    Datafile = fits.open(path)
    hdr = Datafile[0].header
    pixelarea = hdr['PIXAR_SR']
    tocps = hdr['PHOTMJSR']
    exptime = hdr['XPOSURE']
    print(exptime)
    data = (Datafile[0].data/tocps)*exptime
    #creating a list of pixels to fit for readout noise
    satischeck = 1
    while satischeck == 1:
        print(data)
        pixforguassian=[]
        lowlimit = 0.0
        uplimit = float(input("Enter the upper limit of flux density for the guassian"))
        for i in range(len(data)):
            for j in range (len(data[i])):
                if (data[i][j] >= lowlimit) & (data[i][j] <= uplimit):
                    pixforguassian.append(data[i][j])
        pixforguassian = np.array(pixforguassian)
        pixforguassian = np.sort(pixforguassian)
        print("try initial guesses: ")
        print(np.mean(pixforguassian),np.std(pixforguassian))
        #plotting the histogram of the pixel values and the guassian fit curve
        n, bins,_ = plt.hist(pixforguassian, 25, density=True)
        plt.ylabel('Frequency')
        plt.xlabel('Time integrated flux ($N_{electrons}$)')
        plt.title('Frequency in Time Integrated Flux Ranges')
        centerofbins = np.zeros([len(bins) - 1])
        errorsinbins = np.zeros([len(bins) - 1])
        for i in range(len(bins)-1):
            centerofbins[i] = (bins[i]+bins[i+1])/2
            errorsinbins[i] = np.sqrt(n[i])
        initmu, initsig = [float(s) for s in input('Enter initial guesses for mu, and sig ').split()]
        popt, pcov = curve_fit(gauss_f, centerofbins, n,sigma=errorsinbins, p0=[initmu, initsig])
        print(popt)
        print(pcov)
        mu_opt, sig_opt = popt
        x_model = np.linspace(min(bins), max(bins), 1000)
        y_model = gauss_f(x_model, mu_opt, sig_opt)
        plt.plot(x_model, y_model)
        plt.show()
        #checking for satisfactory fit
        satisfactory = input("Is the fit satisfactory?[y/n]")
        #jumping back to take the limit:
        if satisfactory == "n":
            satischeck = 1
            clear_output()
        #proceding with possion noise and total noise calculation:
        elif satisfactory == "y":
            satischeck = 0
    sigma_sky=sig_opt
    data = (data - mu_opt)
    print(sigma_sky)
    #possion(noise:/tocps)*pixelarea
    sigma_p_squared = np.zeros([len(data),len(data[0])])
    for i in range(len(data)):
        for j in range(len(data[0])):
            sigma_p_squared[i][j] = data[i][j]
    sigma_sky_for_total_exp_sq = (sigma_sky)**2
    sigma_tot_sq = sigma_p_squared + sigma_sky_for_total_exp_sq 
    sigma_tot = (np.sqrt(sigma_tot_sq)/exptime)*tocps
    data = data*tocps
    #writing the fits file
    hdu = fits.PrimaryHDU(sigma_tot)
    hdu.writeto(writefile,overwrite=True)
    with fits.open(writefile, mode='update') as hdu:
        hdu[0].header=hdr# Change something in hdul.
        hdu.flush()  # changes are written back to original.fits
    hdu1 = fits.PrimaryHDU(data)
    hdu1.writeto(bkgsubfile,overwrite=True)
    with fits.open(bkgsubfile,mode= 'update') as hdu1:
        hdu1[0].header=hdr# Change something in hdul.
        hdu1.flush() 
    clear_output()

In [4]:
#reading the header file ('change the redundant import of this file above...')
paths = ["jw01355-o015_t001_miri_f560w_i2d-1-~1.fits","jw01355-o015_t001_miri_f770w_i2d-1-~1.fits","jw01355-o015_t001_miri_f1000w_i2d-1-~1.fits","jw01355-o015_t001_miri_f1280w_i2d-1-~1.fits","jw01355-o015_t001_miri_f1500w_i2d-1-~1.fits","jw01355-o015_t001_miri_f1800w_i2d-1-~1.fits","jw01355-o015_t001_miri_f2100w_i2d-1-~1.fits","jw01355-o016_t001_nircam_clear-f115w_i2d-1-~1.fits","jw01355-o016_t001_nircam_clear-f150w_i2d-1-~1.fits","jw01355-o016_t001_nircam_clear-f200w_i2d-1-~1.fits","jw01355-o016_t001_nircam_clear-f277w_i2d-1-~1.fits","jw01355-o016_t001_nircam_clear-f356w_i2d-1-~1.fits","jw01355-o016_t001_nircam_clear-f444w_i2d-1-~1.fits"]
writingfiles = ["miri560noisemap.fits","miri770noisemap.fits","miri1000noisemap.fits","miri1280noisemap.fits","miri1500noisemap.fits","miri1800noisemap.fits","miri2100noisemap.fits","nircam115noisemap.fits","nircam150noisemap.fits","nircam200noisemap.fits","nircam277noisemap.fits","nircam356noisemap.fits","nircam444noisemap.fits"]
bkgfiles = ['miri560bkgsub.fits','miri770bkgsub.fits','miri1000bkgsub.fits','miri1280bkgsub.fits','miri1500bkgsub.fits',"miri1800bkgsum.fits","miri2100bkgsub.fits","nircam115bkgsub.fits","nircam150bkgsub.fits","nircam200bkgsub.fits","nircam277bkgsub.fits","nircam356bkgsub.fits","nircam444bkgsub.fits"]
for i in range(len(paths)):
    print("working on ",paths[i])
    Check_if_worked_on_previously = input("Do you want to obtain a noisemap for this file?[y/n]")
    if Check_if_worked_on_previously == 'n':
        i = i+1
        clear_output()
    elif Check_if_worked_on_previously =='y':
        NoiseMap(paths[i],writingfiles[i],bkgfiles[i])